In [1]:
import gzip

# Define the path to one of the .ent.gz files to examine its contents
ent_gz_file_path = 'pdb/structures/03/pdb103d.ent.gz'

# Read the contents of the .ent.gz file
with gzip.open(ent_gz_file_path, 'rt') as f:
    ent_content = f.read()

# Display a portion of the file content
print(ent_content[:1000])  # Display the first 1000 characters for inspection


HEADER    DNA                                     16-DEC-94   103D              
TITLE     THE UNUSUAL STRUCTURE OF THE HUMAN CENTROMERE (GGA)2 MOTIF: UNPAIRED  
TITLE    2 GUANOSINE RESIDUES STACKED BETWEEN SHEARED G(DOT)A PAIRS             
COMPND    MOL_ID: 1;                                                            
COMPND   2 MOLECULE: DNA (5'-D(*GP*TP*GP*GP*AP*AP*TP*GP*GP*AP*AP*C)-3');        
COMPND   3 CHAIN: A, B;                                                         
COMPND   4 ENGINEERED: YES                                                      
SOURCE    MOL_ID: 1;                                                            
SOURCE   2 SYNTHETIC: YES;                                                      
SOURCE   3 OTHER_DETAILS: CHEMICALLY SYNTHESIZED                                
KEYWDS    DNA, DOUBLE HELIX, G-G STACKING, G:A MISMATCH, HUMAN CENTROMERE       
KEYWDS   2 REPEAT, GA-BRACKETED G-STACK MOTIF                                   
EXPDTA    SOLUTION NMR      

In [6]:
import os
import gzip
from Bio import PDB
from tqdm import tqdm
import datetime
from multiprocessing import Pool, cpu_count

def parse_date(date_str):
    # Attempt to parse the date using different formats
    for date_format in ('%d-%b-%y', '%Y-%m-%d'):
        try:
            return datetime.datetime.strptime(date_str, date_format)
        except ValueError:
            continue
    # If no formats match, raise an error
    raise ValueError(f"Date {date_str} does not match any known format")

def parse_pdb_file_metadata(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    with gzip.open(pdb_file, 'rt') as f:
        structure = parser.get_structure('struct', f)
    
    header = structure.header
    deposition_date = parse_date(header['deposition_date'])
    resolution = header.get('resolution', None)
    experiment_method = header['structure_method']
    
    return deposition_date, resolution, experiment_method

def filter_pdb_file(pdb_file):
    deposition_date, resolution, experiment_method = parse_pdb_file_metadata(pdb_file)

    if deposition_date < datetime.datetime(2020, 5, 1) and experiment_method == 'x-ray diffraction' and resolution and resolution < 9.0:
        return pdb_file
    return None

def filter_pdb_directory(directory, num_workers=cpu_count()):
    pdb_files = []
    filtered_pdb_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.ent.gz'):
                pdb_files.append(os.path.join(root, file))

    with Pool(processes=num_workers) as pool:
        for result in tqdm(pool.imap_unordered(filter_pdb_file, pdb_files), total=len(pdb_files)):
            if result:
                filtered_pdb_files.append(result)

    return filtered_pdb_files

# Directory containing pdb/structures
pdb_directory = 'pdb/structures'

# Filter the PDB structures
filtered_pdb_files = filter_pdb_directory(pdb_directory, num_workers=128)

# Print the number of filtered structures
print("Total structures matching criteria:", len(filtered_pdb_files))


100%|██████████| 209282/209282 [07:28<00:00, 466.86it/s]

Total structures matching criteria: 150853


In [17]:
import os
import gzip
from Bio import PDB
from tqdm import tqdm
import pandas as pd
from multiprocessing import Pool, cpu_count

def parse_pdb_file(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    with gzip.open(pdb_file, 'rt') as f:
        structure = parser.get_structure('struct', f)
        return structure

def analyze_structure(structure):
    gaps = []
    for model in structure:
        for chain in model:
            res_numbers = [res.id[1] for res in chain.get_residues() if PDB.is_aa(res)]
            gaps_in_chain = [res_numbers[i+1] - res_numbers[i] - 1 for i in range(len(res_numbers) - 1) if res_numbers[i+1] - res_numbers[i] > 1]
            gaps.extend(gaps_in_chain)
    return gaps

def process_pdb_file(pdb_file):
    structure = parse_pdb_file(pdb_file)
    gaps = analyze_structure(structure)
    return pdb_file, gaps

def analyze_pdb_directory(directory, num_workers=cpu_count()):
    all_gaps = []
    pdb_files_with_gaps = []
    pdb_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.ent.gz'):
                pdb_files.append(os.path.join(root, file))

    with Pool(processes=num_workers) as pool:
        for pdb_file, gaps in tqdm(pool.imap_unordered(process_pdb_file, pdb_files), total=len(pdb_files)):
            if gaps:
                pdb_files_with_gaps.append(pdb_file)
                all_gaps.extend(gaps)

    return pdb_files, pdb_files_with_gaps, all_gaps

def summarize_gaps(pdb_files, pdb_files_with_gaps, all_gaps):
    total_structures = len(set(pdb_files))
    structures_with_gaps = len(set(pdb_files_with_gaps))
    percent_with_gaps = (structures_with_gaps / total_structures) * 100
    gap_summary = pd.Series(all_gaps).describe()

    return {
        'total_structures': total_structures,
        'structures_with_gaps': structures_with_gaps,
        'percent_with_gaps': percent_with_gaps,
        'gap_summary': gap_summary
    }

# Directory containing pdb/structures
pdb_directory = 'pdb/structures'

# Analyze the structures
pdb_files, pdb_files_with_gaps, all_gaps = analyze_pdb_directory(pdb_directory, num_workers=128)

# Summarize the results
summary = summarize_gaps(pdb_files, pdb_files_with_gaps, all_gaps)

# Print summary
print("Total structures analyzed:", summary['total_structures'])
print("Structures with gaps:", summary['structures_with_gaps'])
print("Percentage of structures with gaps:", summary['percent_with_gaps'])
print("\nGap size summary:\n", summary['gap_summary'])


100%|██████████| 209282/209282 [05:54<00:00, 590.68it/s]


Total structures analyzed: 209282
Structures with gaps: 89381
Percentage of structures with gaps: 42.708403016026224

Gap size summary:
 count    425796.000000
mean         28.730594
std         179.025913
min           1.000000
25%           3.000000
50%           7.000000
75%          14.000000
max        9060.000000
dtype: float64


In [ ]:
import os
import gzip
from Bio import PDB
from tqdm import tqdm
import datetime
from multiprocessing import Pool, cpu_count

def parse_date(date_str):
    # Attempt to parse the date using different formats
    for date_format in ('%d-%b-%y', '%Y-%m-%d'):
        try:
            return datetime.datetime.strptime(date_str, date_format)
        except ValueError:
            continue
    # If no formats match, raise an error
    raise ValueError(f"Date {date_str} does not match any known format")

def parse_pdb_file_metadata(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    with gzip.open(pdb_file, 'rt') as f:
        structure = parser.get_structure('struct', f)
    
    header = structure.header
    deposition_date = parse_date(header['deposition_date'])
    resolution = header.get('resolution', None)
    experiment_method = header['structure_method']
    
    return deposition_date, resolution, experiment_method

def filter_pdb_file(pdb_file):
    deposition_date, resolution, experiment_method = parse_pdb_file_metadata(pdb_file)
    
    # Debugging: Print or log the criteria checks
    print(f"Processing {pdb_file}:")
    print(f"  Deposition Date: {deposition_date}")
    print(f"  Resolution: {resolution}")
    print(f"  Experiment Method: {experiment_method}")
    
    if deposition_date < datetime.datetime(2020, 5, 1):
        print("  -> Date check passed")
    else:
        print("  -> Date check failed")

    if experiment_method == 'x-ray diffraction':
        print("  -> Experiment method check passed")
    else:
        print("  -> Experiment method check failed")

    if resolution and resolution < 9.0:
        print("  -> Resolution check passed")
    else:
        print("  -> Resolution check failed or missing")
    
    # import pdb
    # pdb.set_trace()
    
    # Combine all checks
    if (deposition_date < datetime.datetime(2020, 5, 1) and 
        experiment_method == 'X-RAY DIFFRACTION' and 
        resolution and resolution < 9.0):
        return pdb_file
    return None

def filter_pdb_directory(directory, num_workers=cpu_count()):
    pdb_files = []
    filtered_pdb_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.ent.gz'):
                pdb_files.append(os.path.join(root, file))

    with Pool(processes=num_workers) as pool:
        for result in tqdm(pool.imap_unordered(filter_pdb_file, pdb_files), total=len(pdb_files)):
            if result:
                filtered_pdb_files.append(result)

    return filtered_pdb_files

# Directory containing pdb/structures
pdb_directory = 'pdb/structures'

# Filter the PDB structures
filtered_pdb_files = filter_pdb_directory(pdb_directory, num_workers=1)

# Print the number of filtered structures
print("Total structures matching criteria:", len(filtered_pdb_files))
